## csvのOhlcvデータをinfluxに追加する

主に [Dukascopy](https://www.dukascopy.jp/marketwatch/historical/)からもってくることを想定している

In [1]:
import os
import pandas as pd
import numpy as np
from influxdb_client.client.influxdb_client_async import InfluxDBClientAsync
from blueOcean.ohlcv import Ohlcv, OhlcvRepository, BUCKET_NAME


In [2]:
INFLUXDB_URL = os.getenv("INFLUXDB_URL")
INFLUXDB_ORG = os.getenv("INFLUXDB_ORG")
INFLUXDB_TOKEN = os.getenv("INFLUXDB_TOKEN")
SOURCE = "Dukascopy"
SYMBOL = "USDJPY"
TIMEFRAME = 1

In [4]:
client = InfluxDBClientAsync(url=INFLUXDB_URL, token=INFLUXDB_TOKEN, org=INFLUXDB_ORG)
repository = OhlcvRepository(client)
df = pd.read_csv("../data/USDJPY_Candlestick_1_m_BID_01.01.2019-01.01.2022.csv")

df["time"] = pd.to_datetime(df["Gmt time"], format="%d.%m.%Y %H:%M:%S.%f", utc=True)
df["source"] = SOURCE
df["symbol"] = SYMBOL
df = df.drop("Gmt time", axis=1)
df.columns = df.columns.str.lower()
ohlcvs = Ohlcv.from_dataframe(df)
for chunk in np.array_split(ohlcvs, 100):
    await repository.save(chunk)

print("--- データインポート完了 ---")

/workspace/blueOcean/ohlcv.py:128: RuntimeWarning: coroutine 'OhlcvRepository.create_bucket_if_not_exists' was never awaited
  self.create_bucket_if_not_exists()
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7e84a4610830>


--- データインポート完了 ---
